In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent,Task,Crew,LLM

In [3]:
from dotenv import  load_dotenv
import os
load_dotenv()

api_key = os.getenv("GEMINI_API_KEY")
serper_api_key=os.getenv("SERPER_API_KEY")

os.environ["OPENAI_API_KEY"] = "dummy"

llm = LLM(model="gemini/gemini-2.5-flash",
          provider="gemini",
          temperature=0.7,
          api_key=api_key)

In [4]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True,
    llm=llm,
)

In [5]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True,
    llm=llm,
)

In [6]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [7]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

In [8]:
from crewai.tools import BaseTool

In [9]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")

    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [10]:
sentiment_analysis_tool = SentimentAnalysisTool()

In [11]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

In [12]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent
)

In [13]:
crew = Crew(
    agents=[sales_rep_agent,
            lead_sales_rep_agent],

    tasks=[lead_profiling_task,
           personalized_outreach_task],

    verbose=True,
    manager_llm=llm,
	memory=True
)

In [14]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: cf966b8b-f79b-48b9-9cd9-aab18da52f59                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

ERROR:root:Error during short_term search: APIStatusError.__init__() missing 2 required keyword-only arguments: 
'response' and 'body'
Traceback (most recent call last):
  File 
"/Users/charith/Desktop/AGENT-LEARN/CREWAI-PRJ/ecommerse-recommender-bots/.venv/lib/python3.10/site-packages/chroma
db/api/models/CollectionCommon.py", line 90, in wrapper
    return func(self, *args, **kwargs)
  File 
"/Users/charith/Desktop/AGENT-LEARN/CREWAI-PRJ/ecommerse-recommender-bots/.venv/lib/python3.10/site-packages/chroma
db/api/models/CollectionCommon.py", line 301, in _validate_and_prepare_query_request
    request_embeddings = self._embed_record_set(record_set=query_records)
  File 
"/Users/charith/Desktop/AGENT-LEARN/CREWAI-PRJ/ecommerse-recommender-bots/.venv/lib/python3.10/site-packages/chroma
db/api/models/CollectionCommon.py", line 526, in _embed_record_set
    return self._embed(input=record_set[field])  # type: ignore[literal-required]
  File 
"/Users/charith/Desktop/AGENT-LEARN/CREWAI-PRJ/ecommerse-recommender-bots/.venv/lib/python3.10/site-packages/chroma
db/api/models/CollectionCommon.py", line 539, in _embed
    return self._embedding_function(input=input)
  File 
"/Users/charith/Desktop/AGENT-LEARN/CREWAI-PRJ/ecommerse-recommender-bots/.venv/lib/python3.10/site-packages/chroma
db/api/types.py", line 466, in __call__
    result = call(self, input)
  File 
"/Users/charith/Desktop/AGENT-LEARN/CREWAI-PRJ/ecommerse-recommender-bots/.venv/lib/python3.10/site-packages/chroma
db/utils/embedding_functions/openai_embedding_function.py", line 113, in __call__
    embeddings = self._client.create(
  File 
"/Users/charith/Desktop/AGENT-LEARN/CREWAI-PRJ/ecommerse-recommender-bots/.venv/lib/python3.10/site-packages/openai
/resources/embeddings.py", line 132, in create
    return self._post(
  File 
"/Users/charith/Desktop/AGENT-LEARN/CREWAI-PRJ/ecommerse-recommender-bots/.venv/lib/python3.10/site-packages/openai
/_base_client.py", line 1259, in post
    return cast(ResponseT, self.request(cast_to, opts, stream=stream, stream_cls=stream_cls))
  File 
"/Users/charith/Desktop/AGENT-LEARN/CREWAI-PRJ/ecommerse-recommender-bots/.venv/lib/python3.10/site-packages/openai
/_base_client.py", line 1047, in request
    raise self._make_status_error_from_response(err.response) from None
openai.AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: dummy. You can 
find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': 
None, 'code': 'invalid_api_key'}}

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File 
"/Users/charith/Desktop/AGENT-LEARN/CREWAI-PRJ/ecommerse-recommender-bots/.venv/lib/python3.10/site-packages/crewai
/memory/storage/rag_storage.py", line 118, in search
    return client.search(
  File 
"/Users/charith/Desktop/AGENT-LEARN/CREWAI-PRJ/ecommerse-recommender-bots/.venv/lib/python3.10/site-packages/crewai
/rag/chromadb/client.py", line 418, in search
    results: QueryResult = collection.query(
  File 
"/Users/charith/Desktop/AGENT-LEARN/CREWAI-PRJ/ecommerse-recommender-bots/.venv/lib/python3.10/site-packages/chroma
db/api/models/Collection.py", line 210, in query
    query_request = self._validate_and_prepare_query_request(
  File 
"/Users/charith/Desktop/AGENT-LEARN/CREWAI-PRJ/ecommerse-recommender-bots/.venv/lib/python3.10/site-packages/chroma
db/api/models/CollectionCommon.py", line 93, in wrapper
    raise type(e)(msg).with_traceback(e.__traceback__)
TypeError: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'

ERROR:root:Error during entities search: APIStatusError.__init__() missing 2 required keyword-only arguments: 
'response' and 'body'
Traceback (most recent call last):
  File 
"/Users/charith/Desktop/AGENT-LEARN/CREWAI-PRJ/ecommerse-recommender-bots/.venv/lib/python3.10/site-packages/chroma
db/api/models/CollectionCommon.py", line 90, in wrapper
    return func(self, *args, **kwargs)
  File 
"/Users/charith/Desktop/AGENT-LEARN/CREWAI-PRJ/ecommerse-recommender-bots/.venv/lib/python3.10/site-packages/chroma
db/api/models/CollectionCommon.py", line 301, in _validate_and_prepare_query_request
    request_embeddings = self._embed_record_set(record_set=query_records)
  File 
"/Users/charith/Desktop/AGENT-LEARN/CREWAI-PRJ/ecommerse-recommender-bots/.venv/lib/python3.10/site-packages/chroma
db/api/models/CollectionCommon.py", line 526, in _embed_record_set
    return self._embed(input=record_set[field])  # type: ignore[literal-required]
  File 
"/Users/charith/Desktop/AGENT-LEARN/CREWAI-PRJ/ecommerse-recommender-bots/.venv/lib/python3.10/site-packages/chroma
db/api/models/CollectionCommon.py", line 539, in _embed
    return self._embedding_function(input=input)
  File 
"/Users/charith/Desktop/AGENT-LEARN/CREWAI-PRJ/ecommerse-recommender-bots/.venv/lib/python3.10/site-packages/chroma
db/api/types.py", line 466, in __call__
    result = call(self, input)
  File 
"/Users/charith/Desktop/AGENT-LEARN/CREWAI-PRJ/ecommerse-recommender-bots/.venv/lib/python3.10/site-packages/chroma
db/utils/embedding_functions/openai_embedding_function.py", line 113, in __call__
    embeddings = self._client.create(
  File 
"/Users/charith/Desktop/AGENT-LEARN/CREWAI-PRJ/ecommerse-recommender-bots/.venv/lib/python3.10/site-packages/openai
/resources/embeddings.py", line 132, in create
    return self._post(
  File 
"/Users/charith/Desktop/AGENT-LEARN/CREWAI-PRJ/ecommerse-recommender-bots/.venv/lib/python3.10/site-packages/openai
/_base_client.py", line 1259, in post
    return cast(ResponseT, self.request(cast_to, opts, stream=stream, stream_cls=stream_cls))
  File 
"/Users/charith/Desktop/AGENT-LEARN/CREWAI-PRJ/ecommerse-recommender-bots/.venv/lib/python3.10/site-packages/openai
/_base_client.py", line 1047, in request
    raise self._make_status_error_from_response(err.response) from None
openai.AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: dummy. You can 
find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': 
None, 'code': 'invalid_api_key'}}

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File 
"/Users/charith/Desktop/AGENT-LEARN/CREWAI-PRJ/ecommerse-recommender-bots/.venv/lib/python3.10/site-packages/crewai
/memory/storage/rag_storage.py", line 118, in search
    return client.search(
  File 
"/Users/charith/Desktop/AGENT-LEARN/CREWAI-PRJ/ecommerse-recommender-bots/.venv/lib/python3.10/site-packages/crewai
/rag/chromadb/client.py", line 418, in search
    results: QueryResult = collection.query(
  File 
"/Users/charith/Desktop/AGENT-LEARN/CREWAI-PRJ/ecommerse-recommender-bots/.venv/lib/python3.10/site-packages/chroma
db/api/models/Collection.py", line 210, in query
    query_request = self._validate_and_prepare_query_request(
  File 
"/Users/charith/Desktop/AGENT-LEARN/CREWAI-PRJ/ecommerse-recommender-bots/.venv/lib/python3.10/site-packages/chroma
db/api/models/CollectionCommon.py", line 93, in wrapper
    raise type(e)(msg).with_traceback(e.__traceback__)
TypeError: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Sales Representative                                                                                    │
│                                                                                                                 │
│  Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that    │
│  recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile,   │
│  focusing on key decision-makers, recent business developments, and potential needs that align with our         │
│  offerings. This task is crucial for tailoring our engagement strategy effectively.                             │
│  Don't make assumptions and only use information you absolutely sure about.                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Sales Representative                                                                                    │
│                                                                                                                 │
│  Thought: Action: List files in directory                                                                       │
│                                                                                                                 │
│  Using Tool: List files in directory                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Lead Sales Representative                                                                               │
│                                                                                                                 │
│  Task: Using the insights gathered from the lead profiling report on DeepLearningAI, craft a personalized       │
│  outreach campaign aimed at Andrew Ng, the CEO of DeepLearningAI. The campaign should address their recent      │
│  product launch and how our solutions can support their goals. Your communication must resonate with            │
│  DeepLearningAI's company culture and values, demonstrating a deep understanding of their business and needs.   │
│  Don't make assumptions and only use information you absolutely sure about.                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Lead Sales Representative                                                                               │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Here is a series of personalized email drafts tailored to DeepLearningAI, specifically targeting Andrew Ng,    │
│  designed to resonate with their company culture and values.                                                    │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  **Email 1: Initial Outreach - Focus on the "Worldwide Agentic AI Course"**                                     │
│                                                                                                                 │
│  **Subject: A Powerful Complement to DeepLearning.AI's Agentic AI Curriculum**                                  │
│                                                                                                                 │
│  Dear Andrew,                                                                                                   │
│                                                                                                                 │
│  I've long admired DeepLearning.AI's unwavering commitment to making world-class AI education accessible        │
│  globally, a mission you've championed with such profound impact. Your recent launch of the 'Worldwide Agentic  │
│  AI Course,' personally taught by you, particularly caught my attention. It's a testament to your foresight in  │
│  equipping the next generation of AI builders with critical skills in designing and evaluating multi-step AI    │
│  agents.                                                                                                        │
│                                                                                                                 │
│  At CrewAI, we've developed a multi-AI Agent framework that we believe offers a unique and powerful complement  │
│  to the practical aspects of your Agentic AI curriculum. Our framework allows for the creation of specialized   │
│  AI personas that collaborate effectively, think critically, and perform complex tasks, providing an ideal      │
│  sandbox for students to directly apply the concepts learned in your course. Imagine your students not just     │
│  learning about multi-step AI agents, but actively building, testing, and refining them within a robust,        │
│  real-world-ready environment.                                                                                  │
│                                                                                                                 │
│  We see a significant opportunity for CrewAI to enhance the hands-on learning experiences within your new       │
│  course, turning theoretical knowledge into immediate, practical application. This could help students gain     │
│  invaluable experience in building and managing the exact types of AI agents you're teaching them to design.    │
│                                                                                                                 │
│  Would you be open to a brief 15-minute call next week to explore how CrewAI could seamlessly integrate into    │
│  your educational offerings and further empower your st

ERROR:root:Error during short_term save: APIStatusError.__init__() missing 2 required keyword-only arguments: 
'response' and 'body'
Traceback (most recent call last):
  File 
"/Users/charith/Desktop/AGENT-LEARN/CREWAI-PRJ/ecommerse-recommender-bots/.venv/lib/python3.10/site-packages/chroma
db/api/models/CollectionCommon.py", line 90, in wrapper
    return func(self, *args, **kwargs)
  File 
"/Users/charith/Desktop/AGENT-LEARN/CREWAI-PRJ/ecommerse-recommender-bots/.venv/lib/python3.10/site-packages/chroma
db/api/models/CollectionCommon.py", line 406, in _validate_and_prepare_upsert_request
    upsert_embeddings = self._embed_record_set(record_set=upsert_records)
  File 
"/Users/charith/Desktop/AGENT-LEARN/CREWAI-PRJ/ecommerse-recommender-bots/.venv/lib/python3.10/site-packages/chroma
db/api/models/CollectionCommon.py", line 526, in _embed_record_set
    return self._embed(input=record_set[field])  # type: ignore[literal-required]
  File 
"/Users/charith/Desktop/AGENT-LEARN/CREWAI-PRJ/ecommerse-recommender-bots/.venv/lib/python3.10/site-packages/chroma
db/api/models/CollectionCommon.py", line 539, in _embed
    return self._embedding_function(input=input)
  File 
"/Users/charith/Desktop/AGENT-LEARN/CREWAI-PRJ/ecommerse-recommender-bots/.venv/lib/python3.10/site-packages/chroma
db/api/types.py", line 466, in __call__
    result = call(self, input)
  File 
"/Users/charith/Desktop/AGENT-LEARN/CREWAI-PRJ/ecommerse-recommender-bots/.venv/lib/python3.10/site-packages/chroma
db/utils/embedding_functions/openai_embedding_function.py", line 113, in __call__
    embeddings = self._client.create(
  File 
"/Users/charith/Desktop/AGENT-LEARN/CREWAI-PRJ/ecommerse-recommender-bots/.venv/lib/python3.10/site-packages/openai
/resources/embeddings.py", line 132, in create
    return self._post(
  File 
"/Users/charith/Desktop/AGENT-LEARN/CREWAI-PRJ/ecommerse-recommender-bots/.venv/lib/python3.10/site-packages/openai
/_base_client.py", line 1259, in post
    return cast(ResponseT, self.request(cast_to, opts, stream=stream, stream_cls=stream_cls))
  File 
"/Users/charith/Desktop/AGENT-LEARN/CREWAI-PRJ/ecommerse-recommender-bots/.venv/lib/python3.10/site-packages/openai
/_base_client.py", line 1047, in request
    raise self._make_status_error_from_response(err.response) from None
openai.AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: dummy. You can 
find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': 
None, 'code': 'invalid_api_key'}}

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File 
"/Users/charith/Desktop/AGENT-LEARN/CREWAI-PRJ/ecommerse-recommender-bots/.venv/lib/python3.10/site-packages/crewai
/memory/storage/rag_storage.py", line 98, in save
    client.add_documents(collection_name=collection_name, documents=[document])
  File 
"/Users/charith/Desktop/AGENT-LEARN/CREWAI-PRJ/ecommerse-recommender-bots/.venv/lib/python3.10/site-packages/crewai
/rag/chromadb/client.py", line 320, in add_documents
    collection.upsert(
  File 
"/Users/charith/Desktop/AGENT-LEARN/CREWAI-PRJ/ecommerse-recommender-bots/.venv/lib/python3.10/site-packages/chroma
db/api/models/Collection.py", line 334, in upsert
    upsert_request = self._validate_and_prepare_upsert_request(
  File 
"/Users/charith/Desktop/AGENT-LEARN/CREWAI-PRJ/ecommerse-recommender-bots/.venv/lib/python3.10/site-packages/chroma
db/api/models/CollectionCommon.py", line 93, in wrapper
    raise type(e)(msg).with_traceback(e.__traceback__)
TypeError: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: e32ff1ac-1b36-465e-b626-32999b5fd40d                                                                     │
│  Agent: Lead Sales Representative                                                                               │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: cf966b8b-f79b-48b9-9cd9-aab18da52f59                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Here is a series of personalized email drafts tailored to DeepLearningAI, specifically           │
│  targeting Andrew Ng, designed to resonate with their company culture and values.                               │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  **Email 1: Initial Outreach - Focus on the "Worldwide Agentic AI Course"**                                     │
│                                                                                                                 │
│  **Subject: A Powerful Complement to DeepLearning.AI's Agentic AI Curriculum**                                  │
│                                                                                                                 │
│  Dear Andrew,                                                                                                   │
│                                                                                                                 │
│  I've long admired DeepLearning.AI's unwavering commitment to making world-class AI education accessible        │
│  globally, a mission you've championed with such profound impact. Your recent launch of the 'Worldwide Agentic  │
│  AI Course,' personally taught by you, particularly caught my attention. It's a testament to your foresight in  │
│  equipping the next generation of AI builders with critical skills in designing and evaluating multi-step AI    │
│  agents.                                                                                                        │
│                                                                                                                 │
│  At CrewAI, we've developed a multi-AI Agent framework that we believe offers a unique and powerful complement  │
│  to the practical aspects of your Agentic AI curriculum. Our framework allows for the creation of specialized   │
│  AI personas that collaborate effectively, think critically, and perform complex tasks, providing an ideal      │
│  sandbox for students to directly apply the concepts learned in your course. Imagine your students not just     │
│  learning about multi-step AI agents, but actively building, testing, and refining them within a robust,        │
│  real-world-ready environment.                                                                                  │
│                                                                                                                 │
│  We see a significant opportunity for CrewAI to enhance the hands-on learning experiences within your new       │
│  course, turning theoretical knowledge into immediate, practical application. This could help students gain     │
│  invaluable experience in building and managing the exact types of AI agents you're teaching them to design.    │
│                                                                                                                 │
│  Would you be open to a brief 15-minute call next week